<a href="https://colab.research.google.com/github/KairaNithin/TPU-Repos/blob/main/TPU_and_GPU_with_perfection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***With GPU***

In [2]:
!pip install datasets
from datasets import load_dataset

In [3]:
mnli = load_dataset('glue', 'mnli')

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [4]:
!pip install -U sentence-transformers
!pip install transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (1.2.0)


In [4]:
import torch
import torch.nn as nn
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer
from torch.utils.data import DataLoader,RandomSampler



import random
import pickle
import re
import time
import datetime

import pandas as pd
import numpy as np


from sklearn.metrics import classification_report

from transformers import get_linear_schedule_with_warmup


import torch.nn as nn
from sklearn.metrics import f1_score


from tqdm import tqdm 
import gc
import os

In [16]:
model = DistilBertForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")
tokenizer = DistilBertTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [12]:
class create_dataset(torch.utils.data.Dataset):
  def __init__(self,dataset):
    self.dataset = dataset
  def __len__(self):
    return(len(self.dataset))
  def __getitem__(self,idx):
    self.inputs = self.dataset[idx]
    self.premise = self.inputs['premise']
    self.hypothesis = self.inputs['hypothesis']
    self.targets = self.inputs['label']

    self.inputs = tokenizer.encode_plus(
        self.premise,self.hypothesis,add_special_tokens=True,
        max_length = 128,
        pad_to_max_length=True,  # will be removed in future versions
        # padding=True,  # Not Working
        # padding = 'max_length',
        return_attention_mask=True
    )
    return({'input_ids':torch.tensor(self.inputs['input_ids']),
            'attention_mask':torch.tensor(self.inputs['attention_mask']),
            'targets':torch.tensor(self.targets)})

In [17]:
train_data = create_dataset(mnli['train'])
valid_data = create_dataset(mnli['validation_matched'])

train_sampler = RandomSampler(train_data)

train_loader = DataLoader(train_data,batch_size=32,sampler=train_sampler,drop_last=True)
valid_loader = DataLoader(valid_data,batch_size=8,drop_last=True)

In [18]:
class ourmodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = DistilBertForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")
  def forward(self,input_ids,attention_mask):
    outputs = self.model(input_ids,attention_mask)['logits']

    return outputs


In [19]:
for batch in train_loader:
  break

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model = ourmodel()
model()

In [9]:
path = 'model.pt'

In [37]:
import numpy as np
device = torch.device("cuda")
model.to(device)
def modelEvaluate(model,test_loader):
  model.eval()
  all_outputs = []
  all_targets = []
  flat_outputs = []
  flat_targets = []
  for batch in tqdm(test_loader):
    with torch.no_grad():
      outputs = model(batch['input_ids'].to(device),batch['attention_mask'].to(device))
    targets = batch['targets'].to(device)
    all_outputs.append(outputs)
    # all_outputs.append(outputs)
    all_targets.append(targets)
  flat_outputs = [np.argmax(j.to('cpu')) for i in all_outputs for j in i]
  flat_targets = [j.to('cpu') for i in all_targets for j in i]
  return flat_outputs, flat_targets

In [38]:
def make_optim(model, rate = 2e-5):
  return torch.optim.AdamW(model.parameters(),
                lr = rate, # default = 5e-5, using 2e-5
                eps = 1e-8) # default = 1e-8

def train_model(train_dataloader, valid_dataloader, numberOfEpochs = 10, task = 1):
  """ Train Loop for the model """
  scale = 1
 
  total_steps = len(train_dataloader)
  print("Start")

  model = ourmodel()  
  device = torch.device("cuda")
  model.to(device)
  
  loss_function = nn.CrossEntropyLoss().to(device)
  epoch_loss = 0
  batch_accuracy_scores = []
  global_pred = []
  global_label = []

  present_rate = 2e-5
  old_best = -1
  epoch = 0

  while(1):
    # when the learn rate falls below a lower threshold, you stop your training
    # until that moment, march on
    epoch += 1
    print("\nEpoch:", epoch)
    print("Present Rate: " + str(present_rate))
    optimizer = make_optim(model, present_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
    gc.collect()
    model.train()
    epoch_loss = 0
    batch_accuracy_scores = []
    train_data_count = float(len(train_dataloader))

    # to check if performance with default weights
    print("i")
    predictions, true_labels = modelEvaluate(model, valid_dataloader)
    print("o")
    score_now = f1_score(true_labels, predictions, average = 'macro')
    print("Validation Macro: " + str(score_now))

    if (score_now > old_best):
      print("Continuing on track")
      old_best = score_now

      # delete previous best 
      delete_filename = path
      open(delete_filename, 'w').close() # overwrite and make the file blank instead
      os.remove(delete_filename) # delete the blank file from google drive will move the file to bin instead
      torch.save(model.state_dict(), path)

    else:
      print("Backtrack")
      model.load_state_dict(torch.load(path))
      present_rate /= (4 * scale)
      scale *= 4
      if present_rate < 1e-8:
        break

    # For quick eval
    cnt = 0
    # for i, batch in tqdm(enumerate(train_dataloader)):
    for i, batch in enumerate(train_dataloader):
        print("Iter: " + str(cnt + 1))
        # COMMENT OUT THE NEXT 2 LINES IN ACTUAL TRAINING
        # if cnt == 4:
        #   break
        cnt += 1
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['targets'].to(device)

        pred = model(b_input_ids,b_input_mask)
        loss = loss_function(pred.view(-1, 3), b_labels.view(-1))
        with torch.no_grad():
          epoch_loss += (loss.item() * len(b_labels))
          global_pred.append(pred)
          global_label.append(b_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

  return model

In [ ]:
model = train_model(train_loader,valid_loader)

Start


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


  0%|          | 0/1226 [00:00<?, ?it/s]


Epoch: 1
Present Rate: 2e-05
i


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  0%|          | 2/1226 [00:00<01:51, 11.00it/s]

  0%|          | 4/1226 [00:00<01:46, 11.43it/s]

  0%|          | 6/1226 [00:00<01:43, 11.82it/s]

  1%|          | 8/1226 [00:00<01:39, 12.22it/s]

  1%|          | 10/1226 [00:00<01:36, 12.62it/s]

  1%|          | 12/1226 [00:00<01:33, 13.01it/s]

  1%|          | 14/1226 [00:01<01:30, 13.36it/s]

  1%|▏         | 16/1226 [00:01<01:30, 13.41it/s]

  1%|▏         | 18/1226 [00:01<01:29, 13.49it/s]

  2%|▏         |

Streaming output truncated to the last 5000 lines.
Iter: 7272
Iter: 7273
Iter: 7274
Iter: 7275
Iter: 7276
Iter: 7277
Iter: 7278
Iter: 7279
Iter: 7280
Iter: 7281
Iter: 7282
Iter: 7283
Iter: 7284
Iter: 7285
Iter: 7286
Iter: 7287
Iter: 7288
Iter: 7289
Iter: 7290
Iter: 7291
Iter: 7292
Iter: 7293
Iter: 7294
Iter: 7295
Iter: 7296
Iter: 7297
Iter: 7298
Iter: 7299
Iter: 7300
Iter: 7301
Iter: 7302
Iter: 7303
Iter: 7304
Iter: 7305
Iter: 7306
Iter: 7307
Iter: 7308
Iter: 7309
Iter: 7310
Iter: 7311
Iter: 7312
Iter: 7313
Iter: 7314
Iter: 7315
Iter: 7316
Iter: 7317
Iter: 7318
Iter: 7319
Iter: 7320
Iter: 7321
Iter: 7322
Iter: 7323
Iter: 7324
Iter: 7325
Iter: 7326
Iter: 7327
Iter: 7328
Iter: 7329
Iter: 7330
Iter: 7331
Iter: 7332
Iter: 7333
Iter: 7334
Iter: 7335
Iter: 7336
Iter: 7337
Iter: 7338
Iter: 7339
Iter: 7340
Iter: 7341
Iter: 7342
Iter: 7343
Iter: 7344
Iter: 7345
Iter: 7346
Iter: 7347
Iter: 7348
Iter: 7349
Iter: 7350
Iter: 7351
Iter: 7352
Iter: 7353
Iter: 7354
Iter: 7355
Iter: 7356
Iter: 7357
Ite



  0%|          | 0/1226 [00:00<?, ?it/s]

  0%|          | 2/1226 [00:00<01:27, 14.06it/s]


Epoch: 2
Present Rate: 2e-05
i




  0%|          | 4/1226 [00:00<01:26, 14.13it/s]

  0%|          | 6/1226 [00:00<01:26, 14.14it/s]

  1%|          | 8/1226 [00:00<01:25, 14.31it/s]

  1%|          | 10/1226 [00:00<01:25, 14.20it/s]

  1%|          | 12/1226 [00:00<01:25, 14.27it/s]

  1%|          | 14/1226 [00:00<01:23, 14.43it/s]

  1%|▏         | 16/1226 [00:01<01:23, 14.48it/s]

  1%|▏         | 18/1226 [00:01<01:23, 14.49it/s]

  2%|▏         | 20/1226 [00:01<01:24, 14.30it/s]

  2%|▏         | 22/1226 [00:01<01:24, 14.25it/s]

  2%|▏         | 24/1226 [00:01<01:23, 14.47it/s]

  2%|▏         | 26/1226 [00:01<01:23, 14.36it/s]

  2%|▏         | 28/1226 [00:01<01:23, 14.36it/s]

  2%|▏         | 30/1226 [00:02<01:23, 14.32it/s]

  3%|▎         | 32/1226 [00:02<01:23, 14.37it/s]

  3%|▎         | 34/1226 [00:02<01:22, 14.54it/s]

  3%|▎         | 36/1226 [00:02<01:22, 14.44it/s]

  3%|▎         | 38/1226 [00:02<01:22, 14.43it/s]

  3%|▎         | 40/1226 [00:02<01:23, 14.27it/s]

  3%|▎         | 42/1226 [00:02<

o
Validation Macro: 0.8004978789217426
Backtrack
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7
Iter: 8
Iter: 9
Iter: 10
Iter: 11
Iter: 12
Iter: 13
Iter: 14
Iter: 15
Iter: 16
Iter: 17
Iter: 18
Iter: 19
Iter: 20
Iter: 21
Iter: 22
Iter: 23
Iter: 24
Iter: 25
Iter: 26
Iter: 27
Iter: 28
Iter: 29
Iter: 30
Iter: 31
Iter: 32
Iter: 33
Iter: 34
Iter: 35
Iter: 36
Iter: 37
Iter: 38
Iter: 39
Iter: 40
Iter: 41
Iter: 42
Iter: 43
Iter: 44
Iter: 45
Iter: 46
Iter: 47
Iter: 48
Iter: 49
Iter: 50
Iter: 51
Iter: 52
Iter: 53
Iter: 54
Iter: 55
Iter: 56
Iter: 57
Iter: 58
Iter: 59
Iter: 60
Iter: 61
Iter: 62
Iter: 63
Iter: 64
Iter: 65
Iter: 66
Iter: 67
Iter: 68
Iter: 69
Iter: 70
Iter: 71
Iter: 72
Iter: 73
Iter: 74
Iter: 75
Iter: 76
Iter: 77
Iter: 78
Iter: 79
Iter: 80
Iter: 81
Iter: 82
Iter: 83
Iter: 84
Iter: 85
Iter: 86
Iter: 87
Iter: 88
Iter: 89
Iter: 90
Iter: 91
Iter: 92
Iter: 93
Iter: 94
Iter: 95
Iter: 96
Iter: 97
Iter: 98
Iter: 99
Iter: 100
Iter: 101
Iter: 102
Iter: 103
Iter: 104
Iter: 105
Iter: 106

# ***With TPU*** 

In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
import os

os.environ['XLA_USE_BF16'] = "1"

In [8]:
import torch
import torch.nn as nn
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer
from torch.utils.data import DataLoader,RandomSampler



import random
import pickle
import re
import time
import datetime

import pandas as pd
import numpy as np


from sklearn.metrics import classification_report

from transformers import get_linear_schedule_with_warmup


import torch.nn as nn
from sklearn.metrics import f1_score


from tqdm import tqdm 
import gc
import os

In [9]:

class ourmodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = DistilBertForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")
  def forward(self,input_ids,attention_mask):
    outputs = self.model(input_ids,attention_mask)['logits']

    return outputs

In [10]:

class create_dataset(torch.utils.data.Dataset):
  def __init__(self,dataset):
    self.dataset = dataset
  def __len__(self):
    return(len(self.dataset))
  def __getitem__(self,idx):
    self.inputs = self.dataset[idx]
    self.premise = self.inputs['premise']
    self.hypothesis = self.inputs['hypothesis']
    self.targets = self.inputs['label']

    self.inputs = tokenizer.encode_plus(
        self.premise,self.hypothesis,add_special_tokens=True,
        max_length = 128,
        pad_to_max_length=True,  # will be removed in future versions
        # padding=True,  # Not Working
        # padding = 'max_length',
        return_attention_mask=True
    )
    return({'input_ids':torch.tensor(self.inputs['input_ids']),
            'attention_mask':torch.tensor(self.inputs['attention_mask']),
            'targets':torch.tensor(self.targets)})

In [48]:
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch_xla.distributed.parallel_loader as pl
import time
model = ourmodel()
def map_fn(index, flags):
  ## Setup 

  # Sets a common random seed - both for initialization and ensuring graph is the same
  torch.manual_seed(flags['seed'])

  # Acquires the (unique) Cloud TPU core corresponding to this process's index
  device = xm.xla_device()  


  ## Dataloader construction

  # Creates the transform for the raw Torchvision data
  # See https://pytorch.org/docs/stable/torchvision/models.html for normalization
  # Pre-trained TorchVision models expect RGB (3 x H x W) images
  # H and W should be >= 224
  # Loaded into [0, 1] and normalized as follows:
 

  # Downloads train and test datasets
  # Note: master goes first and downloads the dataset only once (xm.rendezvous)
  #   all the other workers wait for the master to be done downloading.

  # if not xm.is_master_ordinal():
  #   xm.rendezvous('download_only_once')

  # train_dataset = datasets.FashionMNIST(
  #   "/tmp/fashionmnist",
  #   train=True,
  #   download=True,
  #   transform=my_transform)

  # test_dataset = datasets.FashionMNIST(
  #   "/tmp/fashionmnist",
  #   train=False,
  #   download=True,
  #   transform=my_transform)
  
  # if xm.is_master_ordinal():
  #   xm.rendezvous('download_only_once')
  
  # Creates the (distributed) train sampler, which let this process only access
  # its portion of the training dataset.

  train_data = create_dataset(mnli['train'])
  valid_data = create_dataset(mnli['validation_matched'])


  train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_data,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)
  
  test_sampler = torch.utils.data.distributed.DistributedSampler(
    valid_data,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=False)
  
  # Creates dataloaders, which load data in batches
  # Note: test loader is not shuffled or sampled
  train_loader = torch.utils.data.DataLoader(
      train_data,
      batch_size=flags['batch_size'],
      sampler=train_sampler,
      num_workers=flags['num_workers'],
      drop_last=True)

  test_loader = torch.utils.data.DataLoader(
      valid_data,
      batch_size=flags['batch_size'],
      sampler=test_sampler,
      shuffle=False,
      num_workers=flags['num_workers'],
      drop_last=True)
  

  ## Network, optimizer, and loss function creation

  # Creates AlexNet for 10 classes
  # Note: each process has its own identical copy of the model
  #  Even though each model is created independently, they're also
  #  created in the same way.


  # net = torchvision.models.alexnet(num_classes=10).to(device).train()

  model.to(device).train()

  loss_fn = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters())


  ## Trains
  train_start = time.time()
  for epoch in range(flags['num_epochs']):
    print("epoch----------------------------------------{}".format(epoch)) 
    para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)
    for batch_num, batch in enumerate(para_train_loader):
      print("iter{}".format(batch_num))
      # data, targets = batch
      input_ids = batch['input_ids']
      attention_mask = batch['attention_mask']
      targets = batch['targets']

      # Acquires the network's best guesses at each class
      output = model(input_ids,attention_mask)

      # Computes loss
      loss = loss_fn(output, targets)

      # Updates model
      optimizer.zero_grad()
      loss.backward()

      # Note: optimizer_step uses the implicit Cloud TPU context to
      #  coordinate and synchronize gradient updates across processes.
      #  This means that each process's network has the same weights after
      #  this is called.
      # Warning: this coordination requires the actions performed in each 
      #  process are the same. In more technical terms, the graph that
      #  PyTorch/XLA generates must be the same across processes. 
      xm.optimizer_step(optimizer)  # Note: barrier=True not needed when using ParallelLoader

  elapsed_train_time = time.time() - train_start
  print("Process", index, "finished training. Train time was:", elapsed_train_time) 


  # Evaluation
  # Sets net to eval and no grad context 
  model.eval()
  eval_start = time.time()
  with torch.no_grad():
    all_outputs = []
    all_targets = []
    flat_outputs = []
    flat_targets = []

    para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)
    for batch_num, batch in enumerate(para_train_loader):
      input_ids = batch['input_ids']
      attention_mask = batch['attention_mask']
      targets = batch['targets']

      # Acquires the network's best guesses at each class
      output = model(input_ids,attention_mask)
      all_outputs.append(output)
    # all_outputs.append(outputs)
      all_targets.append(targets)
    flat_outputs = [torch.argmax(j.to('cpu')) for i in all_outputs for j in i]
    flat_targets = [j.to('cpu') for i in all_targets for j in i]
    # score = f1_score(flat_outputs,flat_targets)
    
  
  elapsed_eval_time = time.time() - eval_start
  print("Process", index, "finished evaluation. Evaluation time was:", elapsed_eval_time)
  print("f1_score = ",f1_score(flat_outputs,flat_targets))
  # print("Process", index, "guessed", num_correct, "of", total_guesses, "correctly for", num_correct/total_guesses * 100, "% accuracy.")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [49]:
len(mnli['train'])/512

766.99609375

In [50]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from sklearn.metrics import f1_score

In [53]:
# Configures training (and evaluation) parameters
flags={}
flags['batch_size'] = 512
flags['num_workers'] = 2
flags['num_epochs'] = 5
flags['seed'] = 1234
import os

os.environ['XLA_USE_BF16'] = "1"

xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')

train_data =  392702
train_loader len =  192
train_data =  392702
train_loader len =  192
train_data =  392702
train_loader len =  192
train_data =  392702
train_loader len =  192
train_data =  392702
train_loader len =  192
train_data =  392702
train_loader len =  192
train_data =  392702
train_loader len =  192
train_data =  392702
train_loader len =  192
epoch----------------------------------------0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


epoch----------------------------------------0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


epoch----------------------------------------0


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


epoch----------------------------------------0
epoch----------------------------------------0
epoch----------------------------------------0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

iter0
iter0
iter0
iter0
iter0
iter0
iter0
iter0
iter1
iter1
iter1
iter1
iter1
iter1
iter1
iter1


KeyboardInterrupt: ignored

In [23]:
len(train_loader)

12271

In [27]:
len(mnli['train'])/1024

383.498046875

In [52]:
1024/4

256.0